In [1]:
'''
[YouTube URL] → [Extract Transcript]
                      ↓
                  [NER Pipeline]
                      ↓
               [Extract Facts / Claims]
                      ↓
             [Fact-check with Gemini]
                      ↓
         [Optionally: Google Search / SerpAPI]
                      ↓
       [Return verdict + explanation + URL]
'''


'\n[YouTube URL] → [Extract Transcript]\n                      ↓\n                  [NER Pipeline]\n                      ↓\n               [Extract Facts / Claims]\n                      ↓\n             [Fact-check with Gemini]\n                      ↓\n         [Optionally: Google Search / SerpAPI]\n                      ↓\n       [Return verdict + explanation + URL]\n'

In [2]:
!pip install google-generativeai
!pip install youtube-transcript-api
!pip install langchain
!pip install langchain-google-genai
!pip install transformers
# !pip install serpapi
!pip install dateparser
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install -U langchain-community
!pip install duckduckgo-search
!pip install google-search-results

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompat

In [3]:
import os
import dateparser
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.utilities import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
import spacy
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool
import time

2025-05-22 13:47:51.470118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747921671.718585      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747921671.788508      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
os.environ["GOOGLE_API_KEY"] = ""
video_url = "https://www.youtube.com/watch?v=JIbIYCM48to&t=99s"

In [5]:
def get_youtube_transcript(video_url):

    video_id = video_url.split("v=")[-1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    text = " ".join([item['text'] for item in transcript])

    return text
text = get_youtube_transcript(video_url)
print(text)

Amazon web services launched in 2006 with a total of three products storage buckets compute instances and a messaging queue today it offers a mind-numbing 200 and something services and what's most confusing is that many of them appear to do almost the exact same thing it's kind of like shopping at a big grocery store where you have different aisles of product categories filled with things to buy that meet the needs of virtually every developer on the planet in today's video we'll walk down these aisles to gain an understanding of over 50 different AWS products so first let's start with a few that are above my pay grade that you may not know exist if you're building robots you can use robomaker to simulate and test your robots at scale then once your robots are in people's homes you can use iot core to collect data from them update their software and manage them remotely if you happen to have a satellite orbiting Earth you can tap into Amazon's Global Network of antennas to connect dat

In [6]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Load spaCy and Hugging Face NER model
nlp = spacy.load("en_core_web_sm")

from transformers import pipeline
import spacy
import dateparser

# Load models
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")
nlp = spacy.load("en_core_web_sm")

def extract_factual_claims(text, window_size=1):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) >= 5]
    extracted_claims = []

    for idx, sentence_text in enumerate(sentences):
        entities = ner_pipeline(sentence_text)

        if entities:
            # Get surrounding context sentences within the window size
            start_idx = max(0, idx - window_size)
            end_idx = min(len(sentences), idx + window_size + 1)
            context_sentences = sentences[start_idx:end_idx]
            context_text = " ".join(context_sentences)

            claim_info = {
                "sentence": sentence_text,
                "context": context_text,
                "entities": []
            }
            for ent in entities:
                entity_data = {
                    "text": ent['word'],
                    "type": ent['entity_group'],
                    "score": ent['score']
                }
                if ent['entity_group'] == "DATE":
                    parsed_date = dateparser.parse(ent['word'])
                    entity_data["parsed_date"] = str(parsed_date) if parsed_date else None
                claim_info["entities"].append(entity_data)

            extracted_claims.append(claim_info)

    return extracted_claims


# Example
claims = extract_factual_claims(text)
for c in claims[:5]:
    print(c)


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cpu
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'sentence': "Amazon web services launched in 2006 with a total of three products storage buckets compute instances and a messaging queue today it offers a mind-numbing 200 and something services and what's most confusing is that many of them appear to do almost the exact same thing it's kind of like shopping at a big grocery store where you have different aisles of product categories filled with things to buy that meet the needs of virtually every developer on the planet in today's video we'll walk down these aisles to gain an understanding of over 50 different AWS products so first let's start with a few that are above my pay grade that you may not know exist if you're building robots you can use robomaker to simulate and test your robots at scale then once your robots are in people's homes you can use iot core to collect data from them update their software and manage them remotely if you happen to have a satellite orbiting Earth you can tap into Amazon's Global Network of antennas 

# AGENT:

In [85]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash-lite-preview", temperature=0.2)

In [86]:
search = DuckDuckGoSearchRun()
DDSearch = Tool(name="Search", func=search.run, description="Useful for looking up facts or credible URLs"  )

In [111]:
tools = [
    DDSearch
]

In [113]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True
)

In [114]:
def fact_check_claim(claim, context):

    prompt = f"""
    You are a fact-checker with access to this tool:
    - duckduck: Useful for finding credible URLs.
    
    
    Given the following context:
    "{context}"
    
    Fact-check this claim: "{claim}"
    
    Important rules:
    1. Ignore spelling mistakes
    2. If uncertain, return "Unverifiable"
    3. Get me three credible URLs. 
    4. Use the search tool if you can't get URLs on your own.
    
    Final Answer: <respond in JSON like below>
    
    {{
      "verdict": "<Correct | Incorrect | Misleading | Unverifiable>",
      "explanation": "<explanation>","sources": ["<url1>", "<url2>", "<url3>"],"confidence": "<percentage>"}}
    
    """
    
    result = agent.invoke(prompt)
    return result

result = fact_check_claim("Paris is in Italy.", "France is in Europe")
print(result)

{'input': '\n    You are a fact-checker with access to this tool:\n    - duckduck: Useful for finding credible URLs.\n    \n    \n    Given the following context:\n    "France is in Europe"\n    \n    Fact-check this claim: "Paris is in Italy."\n    \n    Important rules:\n    1. Ignore spelling mistakes\n    2. If uncertain, return "Unverifiable"\n    3. Get me three credible URLs. \n    4. Use the search tool if you can\'t get URLs on your own.\n    \n    Final Answer: <respond in JSON like below>\n    \n    {\n      "verdict": "<Correct | Incorrect | Misleading | Unverifiable>",\n      "explanation": "<explanation>","sources": ["<url1>", "<url2>", "<url3>"],"confidence": "<percentage>"}\n    \n    ', 'output': '```json\n{\n  "verdict": "Incorrect",\n  "explanation": "Paris is the capital of France, not Italy.",\n  "sources": [\n    "https://www.britannica.com/place/Paris",\n    "https://www.worldatlas.com/cities/where-is-paris-located.html",\n    "https://www.britannica.com/place/Fr

In [115]:
print(result['output'])

```json
{
  "verdict": "Incorrect",
  "explanation": "Paris is the capital of France, not Italy.",
  "sources": [
    "https://www.britannica.com/place/Paris",
    "https://www.worldatlas.com/cities/where-is-paris-located.html",
    "https://www.britannica.com/place/France"
  ],
  "confidence": "100%"
}
```


In [116]:
import re
import json

def extract_json_from_output(result):
    output_text = result['output']

    # Try to extract JSON block (inside ```json ... ``` or just curly braces)
    match = re.search(r'```json\s*(\{.*?\})\s*```', output_text, re.DOTALL)

    if not match:
        # Fallback: try to find any JSON-looking block without the backticks
        match = re.search(r'(\{.*?\})', output_text, re.DOTALL)

    if match:
        json_text = match.group(1)
        try:
            data = json.loads(json_text)
            return data
        except json.JSONDecodeError as e:
            return {"error": f"Failed to parse JSON: {e}"}
    else:
        return {"error": "No JSON found in output."}

# Example usage
result = {
  'input': '...',
  'output': '```json\n{\n  "verdict": "Incorrect",\n  "explanation": "Paris is the capital of France, not Italy.",\n  "sources": [\n    "https://www.britannica.com/place/Paris",\n    "https://www.worldatlas.com/cities/where-is-paris-located.html",\n    "https://www.tripsavvy.com/is-paris-in-france-or-italy-7506704"\n  ],\n  "confidence": "100%"\n}\n```'
}

parsed_output = extract_json_from_output(result)
print(json.dumps(parsed_output, indent=2))


{
  "verdict": "Incorrect",
  "explanation": "Paris is the capital of France, not Italy.",
  "sources": [
    "https://www.britannica.com/place/Paris",
    "https://www.worldatlas.com/cities/where-is-paris-located.html",
    "https://www.tripsavvy.com/is-paris-in-france-or-italy-7506704"
  ],
  "confidence": "100%"
}


In [122]:
video_url = "https://www.youtube.com/watch?v=9GvgXI5BOws"
transcript = get_youtube_transcript(video_url)
facts = extract_factual_claims(transcript)

for fact in facts:
    print(f"📝 Fact: {fact['sentence']}")
    result = fact_check_claim(fact['sentence'], fact['context'])
    parsed_output = extract_json_from_output(result)
    print(json.dumps(parsed_output, indent=2))
    print("-" * 50)
    time.sleep(10)


📝 Fact: Was God a human invention?


DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit

# LLM Chain:

In [105]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.tools import DuckDuckGoSearchRun
import re, json

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash-lite-preview", temperature=0.2)

# Setup tools
search = DuckDuckGoSearchRun()
tools = {
    "duckduck": search.run
}

# Define prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "claim"],
    template="""
You are a fact-checker with access to this tool:
- duckduck: Useful for finding credible URLs.


Given the following context:
"{context}"

Fact-check this claim: "{claim}"

Important rules:
1. Ignore spelling mistakes
2. If uncertain, return "Unverifiable"
3. Use the search tool if you can't get URLs on your own
Final Answer: <respond in JSON like below>

{{
  "verdict": "<Correct | Incorrect | Misleading | Unverifiable>",
  "explanation": "<explanation>",
  "sources": ["<url1>", "<url2>", "<url3>"],
  "confidence": "<percentage>"
}}

"""
)

# Create LLMChain
llm_chain = LLMChain(llm=llm,prompt=prompt_template)


In [83]:
def fact_check_claim(claim, context):
    result = llm_chain.run(context=context, claim=claim)

    # Get actual text content
    output_text = result.content if hasattr(result, 'content') else str(result)

    # Strip code block markers if present
    output_text = output_text.strip("```json").strip("```").strip()

    try:
        json_result = json.loads(output_text)
        return json_result
    except Exception as e:
        print("Failed to parse JSON:", e)
        print("Raw LLM Output:", output_text)
        return {
            "verdict": "Unclear",
            "explanation": "Failed to parse result.",
            "sources": []
        }

In [84]:
print(fact_check_claim(context="The claim relates to current global temperature trends.", claim="Global temperatures have dropped since 2020."))

{'verdict': 'Incorrect', 'explanation': 'Multiple sources indicate that global temperatures have continued to increase since 2020, not decreased. 2023 was the warmest year on record, and the trend continues upward.', 'sources': ['https://climate.nasa.gov/vital-signs/global-temperature/', 'https://www.ncei.noaa.gov/global/temperature', 'https://www.carbonbrief.org/state-of-the-climate-global-emissions-reach-record-high-in-2023/'], 'confidence': '99'}


# Google Tools

In [40]:
from google.generativeai import GenerativeModel
import google.generativeai # Import the main library

# List available models
# models = GenerativeModel.list() # Incorrect way to list models
models = google.generativeai.list_models() # Correct way to list models
for m in models:
    print(m.name, m.description)

models/embedding-gecko-001 Obtain a distributed representation of a text.
models/gemini-1.0-pro-vision-latest The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
models/gemini-pro-vision The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
models/gemini-1.5-pro-latest Alias that points to the most recent production (non-experimental) release of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens.
models/gemini-1.5-pro-001 Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens, released in May of 2024.
models/gemini-1.5-pro-002 Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens, released in September of 2024.
